In [66]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas
import xarray
import cftime
import tensorflow as tf

In [67]:
datapath = 'nino34_monthly.nc'
nino34 = xarray.open_dataset(datapath, decode_times = False)
print(nino34)
nino34 = np.array(nino34['nino34'].values)

<xarray.Dataset>
Dimensions:         (bounds: 2, month: 12, time: 7800)
Coordinates:
  * time            (time) float64 15.5 45.0 74.5 ... 2.372e+05 2.372e+05
Dimensions without coordinates: bounds, month
Data variables:
    nino34          (time) float64 ...
    time_bnds       (time, bounds) float64 ...
    areacello       float32 ...
    days_per_month  (month) int32 ...


In [68]:
def ONI(nino34, m = 3):
    oni = np.array(nino34)
    length = nino34.shape[0]
    for i in range(length):
        oni[i] = np.mean(nino34[max(0, (i - m + 1)) : min((i + 1), length)])
    return oni

In [69]:
oni = ONI(nino34)

In [70]:
def climatology(nino34):
    clm = np.zeros(12)
    length = nino34.shape[0]
    for month in range(12):
        section = [12 * i + month for i in range(length // 12)]
        clm[month] = np.mean(nino34[section])
    return clm

In [71]:
clm = climatology(nino34)

In [72]:
def SST_anomaly(nino34, clm):
    anm = np.array(nino34)
    length = nino34.shape[0]
    for i in range(length):
        anm[i] = nino34[i] - clm[i % 12]
    return anm

In [73]:
anm = SST_anomaly(nino34, clm)
oanm = ONI(anm)

# Data Preparation

In [74]:
T = 7      # prediction timeline
H = 32   # history used for prediction
include_month = 1           # 1 if we use the month as a feature, 0 otherwise
threshold = 0.5         
signal = np.array(nino34[12 * 50:])   # data used for training/testing
length = signal.shape[0]    # number of data points

mean = np.mean(signal)
std = np.std(signal)

signal = (signal - mean) / std

In [75]:
size = 7000
sample_len = 350
n_samples = size // sample_len

In [76]:
np.random.seed(0)
        
# create the 'history matrix'
data = np.ndarray((n_samples, sample_len, H + include_month))
for i in range(n_samples):
    for j in range(sample_len):
        t = sample_len * i + j
        if(include_month == False):
             data[i,j] = signal[t:(t + H)]
        else:
            data[i,j] = np.append(signal[t:(t + H)], (t + H + T) % 12 / 12)

labels = np.ndarray((n_samples, sample_len), dtype = int)
        
for i in range(n_samples):
    for j in range(sample_len):
        t = sample_len * i + j
        labels[i, j] = oanm[t + H + T]

split = n_samples // 10      
shuffle = np.random.permutation(n_samples)
train_ind = np.array(shuffle[0: 8 * split])
val_ind = np.array(shuffle[8 * split: 9 * split])
test_ind = np.array(shuffle[9 * split: size])

train = np.array(data[train_ind])
train_labels = np.array(labels[train_ind])

val = np.array(data[val_ind])
val_labels = np.array(labels[val_ind])

test = np.array(data[test_ind])
test_labels = np.array(labels[test_ind])

In [101]:
epochs = 100
        
acc = 0
loss = 0


for n_neurons in range(10, 100, 10):
    for l in ['mse', 'mae']:

        model = tf.keras.models.Sequential([
                  tf.keras.layers.LSTM(n_neurons),
                  tf.keras.layers.Dense(1)
                ])

        model.reset_states()

        model.compile(optimizer='adam',
                loss = l,
                metrics=['mse', 'mae'])

        model.fit(train, train_labels, batch_size=n_samples, epochs = epochs, verbose = 0)
        (loss, mse, mae) = model.evaluate(val, val_labels, verbose = 0)

        print(n_neurons, l, ":", mse, mae)

10 mse : 0.4439932703971863 0.4565284252166748
10 mae : 0.28593870997428894 0.3852995038032532
20 mse : 0.2769436836242676 0.3629114627838135
20 mae : 0.2807210683822632 0.3219720125198364
30 mse : 0.34416961669921875 0.38427823781967163
30 mae : 0.32868683338165283 0.31550246477127075
40 mse : 0.3221272826194763 0.36283326148986816
40 mae : 0.27166908979415894 0.3132445216178894
50 mse : 0.32102036476135254 0.30800414085388184
50 mae : 0.357069194316864 0.3703349828720093
60 mse : 0.3003605604171753 0.34306246042251587
60 mae : 0.3151930570602417 0.2767164707183838
70 mse : 0.271484375 0.32294368743896484
70 mae : 0.28286606073379517 0.31111547350883484
80 mse : 0.30231577157974243 0.31452441215515137
80 mae : 0.3346917927265167 0.3304855227470398
90 mse : 0.34859979152679443 0.3579505681991577
90 mae : 0.3212650418281555 0.287781298160553


# Trying Different Data Formats

In [119]:
def generate_data(size, sample_len):
    n_samples = size // sample_len
    np.random.seed(0)
        
    # create the 'history matrix'
    data = np.ndarray((n_samples, sample_len, H + include_month))
    for i in range(n_samples):
        for j in range(sample_len):
            t = sample_len * i + j
            if(include_month == False):
                 data[i,j] = signal[t:(t + H)]
            else:
                data[i,j] = np.append(signal[t:(t + H)], (t + H + T) % 12 / 12)

    labels = np.ndarray((n_samples, sample_len), dtype = int)
        
    for i in range(n_samples):
        for j in range(sample_len):
            t = sample_len * i + j
            labels[i, j] = oanm[t + H + T]

    split = n_samples // 10      
    shuffle = np.random.permutation(n_samples)
    train_ind = np.array(shuffle[0: 8 * split])
    val_ind = np.array(shuffle[8 * split: 9 * split])
    test_ind = np.array(shuffle[9 * split: size])

    train = np.array(data[train_ind])
    train_labels = np.array(labels[train_ind])

    val = np.array(data[val_ind])
    val_labels = np.array(labels[val_ind])

    test = np.array(data[test_ind])
    test_labels = np.array(labels[test_ind])
    
    return (train, train_labels, val, val_labels, test, test_labels) 

In [122]:
epochs = 100
n_neurons = 40

for sample_len in range(175, 700, 175):
    for l in ['mae', 'mse']:
        (train, train_labels, val, val_labels, test, test_labels) = generate_data(7000, sample_len)
        
        n_samples = 7000 // sample_len
        
        model = tf.keras.models.Sequential([
             tf.keras.layers.LSTM(n_neurons),
             tf.keras.layers.Dense(1)
                ])

        model.reset_states()

        model.compile(optimizer='adam',
                loss = l,
                metrics=['mse', 'mae'])

        model.fit(train, train_labels, batch_size=n_samples, epochs = epochs, verbose = 0)
        (loss, mse, mae) = model.evaluate(val, val_labels, verbose = 0)

        print(sample_len, l, ":", mse, mae)

175 mae : 0.20791709423065186 0.23351305723190308
175 mse : 0.2148858904838562 0.25946658849716187
350 mae : 0.36433857679367065 0.35473954677581787
350 mse : 0.34228917956352234 0.3974469304084778
525 mae : 0.3219010829925537 0.4519275724887848
525 mse : 0.23203979432582855 0.2688826322555542


In [125]:
epochs = 100
n_neurons = 40

for sample_len in range(25, 176, 50):
    for l in ['mae', 'mse']:
        (train, train_labels, val, val_labels, test, test_labels) = generate_data(7000, sample_len)
        
        n_samples = 7000 // sample_len
        
        model = tf.keras.models.Sequential([
             tf.keras.layers.LSTM(n_neurons),
             tf.keras.layers.Dense(1)
                ])

        model.reset_states()

        model.compile(optimizer='adam',
                loss = l,
                metrics=['mse', 'mae'])

        model.fit(train, train_labels, batch_size=n_samples, epochs = epochs, verbose = 0)
        (loss, mse, mae) = model.evaluate(val, val_labels, verbose = 0)

        print(sample_len, l, ":", mse, mae)

25 mae : 0.1931416541337967 0.1976475715637207
25 mse : 0.22675611078739166 0.27281227707862854
75 mae : 0.2589319944381714 0.2879652678966522
75 mse : 0.23260360956192017 0.29476746916770935
125 mae : 0.4088299870491028 0.3584330677986145
125 mse : 0.4196997284889221 0.3758447468280792
175 mae : 0.20199579000473022 0.2518816888332367
175 mse : 0.20040751993656158 0.21377389132976532


In [129]:
epochs = 1000
for sample_len in range(1, 11):
    for l in ['mae', 'mse']:
        (train, train_labels, val, val_labels, test, test_labels) = generate_data(7000, sample_len)
        
        n_samples = 7000 // sample_len
        
        model = tf.keras.models.Sequential([
             tf.keras.layers.LSTM(n_neurons),
             tf.keras.layers.Dense(1)
                ])

        model.reset_states()

        model.compile(optimizer='adam',
                loss = l,
                metrics=['mse', 'mae'])

        model.fit(train, train_labels, batch_size=n_samples, epochs = epochs, verbose = 0)
        (loss, mse, mae) = model.evaluate(val, val_labels, verbose = 0)

        print(sample_len, l, ":", mse, mae)

1 mae : 0.2557797520501273 0.2254977747797966
1 mse : 0.3479794868401119 0.42146564160074507
2 mae : 0.22148669157709394 0.1954586889062609
2 mse : 0.4869651378052575 0.4785437212671552
3 mae : 0.3171760131616961 0.26406358128069807
3 mse : 0.6210058873815086 0.5460952113626341
4 mae : 0.2620085006100791 0.21815091337476458
4 mse : 0.5294170721939632 0.5154748010635376
5 mae : 0.3209615869181497 0.2606730282306671
5 mse : 0.6040602488177164 0.5626546110425676
6 mae : 0.2950953882316063 0.25973642077939263
6 mse : 0.6084838201259745 0.5727957857066187
7 mae : 0.21050041675567627 0.22100984811782837
7 mse : 0.365741171836853 0.4507310485839844
8 mae : 0.2969372246799798 0.26481383507964257
8 mse : 0.4747670741601922 0.5010209090408237
9 mae : 0.2874623378375908 0.2657031209824921
9 mse : 0.4010944796072972 0.4619648940377421
10 mae : 0.31363159332956586 0.30367177384240285
10 mse : 0.4688867798873356 0.5035372691495078


# Adding Multiple Fully Connected Layers

In [134]:
epochs = 100
n_neurons = 40
sample_len = 25
n_samples = 7000 // sample_len
(train, train_labels, val, val_labels, test, test_labels) = generate_data(7000, sample_len)

for N in [H, 2 * H, 3 * H, 4 * H]:
    model = tf.keras.models.Sequential([
             tf.keras.layers.LSTM(n_neurons),
             tf.keras.layers.Dense(N),
             tf.keras.layers.Dense(1)
            ])

    model.reset_states()

    model.compile(optimizer='adam', loss = 'mae', metrics=['mse', 'mae'])

    model.fit(train, train_labels, batch_size=n_samples, epochs = epochs, verbose = 0)
    (loss, mse, mae) = model.evaluate(val, val_labels, verbose = 0)

    print(N, mse, mae)

32 0.19299684464931488 0.18327488005161285
64 0.19112162292003632 0.17845840752124786
96 0.19061000645160675 0.17806892096996307
128 0.19148209691047668 0.17133986949920654


In [135]:
# MAE
for N1 in [3 * H, 4 * H, 5 * H, 6 * H]:
    for N2 in [3 * H, 4 * H, 5 * H, 6 * H]:
        model = tf.keras.models.Sequential([
             tf.keras.layers.LSTM(n_neurons),
             tf.keras.layers.Dense(N1),
             tf.keras.layers.Dense(N2),
             tf.keras.layers.Dense(1)
            ])

        model.reset_states()

        model.compile(optimizer='adam', loss = 'mae', metrics=['mse', 'mae'])
    
        model.fit(train, train_labels, batch_size=n_samples, epochs = epochs, verbose = 0)
        (loss, mse, mae) = model.evaluate(val, val_labels, verbose = 0)

        print(N1, N2, mse, mae)

96 96 0.1914338320493698 0.17169751226902008
96 128 0.19182442128658295 0.16956207156181335
96 160 0.19161559641361237 0.17066515982151031
96 192 0.19169606268405914 0.17110373079776764
128 96 0.19167205691337585 0.17395862936973572
128 128 0.1915774643421173 0.1689101606607437
128 160 0.19137586653232574 0.1709912121295929
128 192 0.19163250923156738 0.17017783224582672
160 96 0.19150756299495697 0.1695082187652588
160 128 0.1913897842168808 0.16959922015666962
160 160 0.19144029915332794 0.16979756951332092
160 192 0.19227100908756256 0.17498715221881866
192 96 0.19215954840183258 0.1737329065799713
192 128 0.19164560735225677 0.16979564726352692
192 160 0.19223950803279877 0.17494085431098938
192 192 0.19122233986854553 0.17242108285427094


In [136]:
# MSE
for N1 in [3 * H, 4 * H, 5 * H, 6 * H]:
    for N2 in [3 * H, 4 * H, 5 * H, 6 * H]:
        model = tf.keras.models.Sequential([
             tf.keras.layers.LSTM(n_neurons),
             tf.keras.layers.Dense(N1),
             tf.keras.layers.Dense(N2),
             tf.keras.layers.Dense(1)
            ])

        model.reset_states()

        model.compile(optimizer='adam', loss = 'mse', metrics=['mse', 'mae'])
    
        model.fit(train, train_labels, batch_size=n_samples, epochs = epochs, verbose = 0)
        (loss, mse, mae) = model.evaluate(val, val_labels, verbose = 0)

        print(N1, N2, mse, mae)

96 96 0.24344365298748016 0.31899362802505493
96 128 0.27740979194641113 0.3390839397907257
96 160 0.23378729820251465 0.28796225786209106
96 192 0.2612089216709137 0.3246050775051117
128 96 0.2736087739467621 0.3388022780418396
128 128 0.22609065473079681 0.2659027576446533
128 160 0.2801613211631775 0.34527701139450073
128 192 0.26062971353530884 0.31787484884262085
160 96 0.2606222331523895 0.34885692596435547
160 128 0.3266465961933136 0.39134034514427185
160 160 0.3009677827358246 0.3586064279079437
160 192 0.24563777446746826 0.306509405374527
192 96 0.34183770418167114 0.3709569275379181
192 128 0.26682230830192566 0.3506458103656769
192 160 0.2602820098400116 0.3616555333137512
192 192 0.2504517138004303 0.3343135714530945
